In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
print("Device:", device)

Device: cpu


In [5]:
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    data_set = pickle.load(f)

In [5]:
data_set[0][1]

0

In [6]:
data = []
targets = []
for i in data_set:
  data.append(i[0])
  targets.append(i[1])

In [7]:
targets_array = np.array(targets)
targets_array_reshaped = targets_array.reshape(-1, 1)
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(targets_array_reshaped)

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [8]:
targets_ohe = ohe.transform(targets_array_reshaped)
print(targets_ohe)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [9]:
print(data[-1])
print(targets[-1])
print(targets_ohe[-1])

[112. 112.   0.  64.   0. 112.  34. 144. 146. 112. 112. 179. 112. 112.
  73.   3.  12. 112. 146.  60.  45. 112.  73.  88. 117.   8. 159. 145.
 159. 112.  13.  80.  92. 159. 190. 112. 185. 190.   8.  73. 125.  77.
  12.   0. 117. 190. 185. 112. 125. 125.  45.  41.  92.  28.  92.  12.
 159.   0.  47.  12.  12.  88.  88.   0.  12. 112. 159.   8. 159.   0.
  12. 112. 125.  23.  47.  41. 159. 112.  45.  12.   8. 145.  47.  33.
  33.  33. 125.  73.   8.  13.  92.  88. 159. 120. 124. 185. 185.  77.
 125.  88.  13. 120. 120. 112. 112. 112. 159. 172.  12. 117.  47.  88.
  88.  36.  92.  33. 125.  88.  88.  88.  12. 112. 159.  88. 125. 119.
  47.  88.  92. 159. 185. 112.  77.  33. 125.  65. 125.  33. 125.  88.
  13. 120. 124. 112.  12.   0.   0.   0.  12. 112. 159. 145.   5.   0.
  88. 112.  12.  47.  88.   0.  12. 112. 159. 145.   5.   0.  12. 112.
 125. 125.  28. 159.  28.  38.  12. 117.  47. 114. 153.  88. 159.   8.
   8. 112.  77.  38.  12.   0. 117. 190. 185. 112.  77.  33.  41. 145.
 145. 

In [9]:
from collections import Counter

labels = [label for _, label in data_set]

class_counts = Counter(labels)
classes = []
print("Liczba klas:", len(class_counts))
for class_label, count in class_counts.items():
    classes.append(count)
classes = np.array(classes)
print(classes)

Liczba klas: 5
[1630  478  154  441  236]


In [10]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [11]:
train_indices = np.random.rand(len(data_set))>0.3

In [13]:
train_indices

array([ True,  True,  True, ...,  True,  True,  True])

In [12]:
data_train = []
targets_train = []
data_test = []
targets_test = []
for i in range(len(data_set)):
  if train_indices[i] == True:
    data_train.append(data_set[i][0])
    targets_train.append(targets_ohe[i])
  else:
    data_test.append(data_set[i][0])
    targets_test.append(targets_ohe[i])

In [15]:
# dataset_size = len(data_set)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size


# train_set = VariableLenDataset(data[:train_indices], targets[:train_indices])
# test_set = VariableLenDataset(data[train_indices:], targets[train_indices:])

In [13]:
train_set = VariableLenDataset(data_train, targets_train)
test_set = VariableLenDataset(data_test, targets_test)

In [17]:
print(train_set[2])

(array([ 66., 100., 148., 148., 146.,  64., 146., 148.,  82.,   0.,  82.,
       100.,  34., 132., 180.,  65.,  80.,  81., 131.,  52.,  34.,  52.,
        64.,  52.,   3.,  66., 147.,  20.,   4., 132., 132., 100., 111.,
        74., 110.,  60.,  92.,  65., 100., 189.,  44.,   8.,   5.,  76.,
        31., 159.,   5., 124.,   4.,  12.,  51., 157.,  57.,  31., 183.,
        57.,  65.,  92.,  69., 124., 122.,  79., 110.,  76.,  12.,  12.,
        12.,  12.,   8., 159.,  12., 156., 100., 111.,  52., 121.,  36.,
        47.,  41.,  41.,   8., 172.,  38.,  12.,  78.,  12.,  88.,  47.,
       119.,  20.,  47.,  12., 159.,  20.,  20.,  76.,  60., 110., 132.,
       185., 120.,  45., 110., 110.,  72., 124., 178.,  73.,   8.,  78.,
        12.,  42., 173.,  12., 150.,  84.,  66., 152.,  69.,   8.,  41.,
       159.,   5.,  78.,  44., 180.,  71.,  92., 152.,   6.,  12.,  45.,
        92., 125.,   8., 156.,  12.,  12., 146.,   8., 185.,   0.,  88.,
        92.,  90.,  12.,  12.,  12.,  78.,  92.,  

In [14]:
pad = -100

# def pad_collate(batch, pad_value=pad):
#     xx, yy = zip(*batch)
#     xx = [torch.tensor(x) for x in xx]
#     x_lens = [len(x) for x in xx]
#     yy_pad = torch.tensor(yy)
#     y_lens = [1] * len(yy)
#     # print(xx)
#     # print(yy)

#     xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
#     # yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

#     return xx_pad, yy_pad, x_lens, y_lens

def pad_collate(batch, pad_value=pad):
    try:
        xx, yy = zip(*batch)
    except ValueError as e:
        print(f"Error while unpacking batch: {e}")
        print(f"Batch content: {batch}")
        raise

    xx = [torch.tensor(x) for x in xx]
    x_lens = [len(x) for x in xx]
    yy_pad = torch.tensor(yy)
    y_lens = [1] * len(yy)

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [15]:
batch_size=256

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [20]:
# dataset_size = len(data)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size

# train_set, test_set = random_split(data, [train_size, test_size])

# batch_size=4
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [21]:
data_set[0]


(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0)

In [22]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

4756
5322


In [23]:
next(iter(train_loader))

<ipython-input-18-76b5b91029d1>:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  yy_pad = torch.tensor(yy)


(tensor([[  80.,  162.,   33.,  ...,  111.,  137.,   64.],
         [  -1.,   -1.,  144.,  ..., -100., -100., -100.],
         [  -1.,   -1.,   -1.,  ..., -100., -100., -100.],
         [ 144.,    8.,  190.,  ..., -100., -100., -100.]], dtype=torch.float64),
 tensor([[0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.]], dtype=torch.float64),
 [308, 288, 68, 270],
 [1, 1, 1, 1])

In [24]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(320, out_size)
        # self.fc = nn.Linear(hidden_size*90*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        # x = torch.transpose(x,0,1)
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        # all_outputs = torch.transpose(all_outputs,0,1)
        # out = torch.flatten(all_outputs,1)
        # output, _ = pad_packed_sequence(all_outputs, batch_first=True)
        # output = output[:, -1, :]

        h_n, _ = hidden
        h_n = torch.flatten(h_n, 1)  # Flatten the hidden state
        # print(h_n[0])
        x = self.fc(h_n)

        # hidden = torch.flatten(hidden,1)
        # print(hidden[0])
        # x = self.fc(hidden)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=320, out_features=5, bias=True)
)

In [25]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size * self.bidirectional, out_size)


    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        h_n, _ = hidden
        # h_n = torch.flatten(h_n, 1)
        # print(h_n)
        x = self.fc(h_n)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)

In [16]:
class LSTMRegressor(nn.Module): #Coś nie tak

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size * self.bidirectional, 10)
        self.fc2 = nn.Linear(10,out_size)


    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        h_n, _ = hidden
        # h_n = torch.flatten(h_n, 1)
        # print(h_n)
        x = F.relu(self.fc(h_n[-1]))
        x = F.tanh(self.fc2(x))
        # x = F.softmax(x, dim=1)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=5, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
)

In [17]:
# lr=0.001 nie zbiega?
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

In [18]:
model = LSTMRegressor(1,5,2,5).to(device)
num_epochs = 1000

for epoch in range(num_epochs):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)

        x = torch.transpose(x, 0, 1)
        print(x)
        preds, _ = model(x, (hidden, state))
        # print(preds.size())
        # preds = torch.transpose(preds, 0, 1)


        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        optimizer.zero_grad()
        # mask = targets != pad
        # print(preds.size())
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

<ipython-input-14-76b5b91029d1>:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  yy_pad = torch.tensor(yy)


tensor([[[  -1.],
         [   0.],
         [  12.],
         ...,
         [  80.],
         [  67.],
         [   0.]],

        [[  -1.],
         [  92.],
         [  12.],
         ...,
         [ 172.],
         [  88.],
         [  67.]],

        [[  -1.],
         [  13.],
         [  78.],
         ...,
         [  13.],
         [ 112.],
         [  73.]],

        ...,

        [[-100.],
         [-100.],
         [-100.],
         ...,
         [-100.],
         [-100.],
         [-100.]],

        [[-100.],
         [-100.],
         [-100.],
         ...,
         [-100.],
         [-100.],
         [-100.]],

        [[-100.],
         [-100.],
         [-100.],
         ...,
         [-100.],
         [-100.],
         [-100.]]], dtype=torch.float64)
tensor([[[  -1.],
         [   3.],
         [  -1.],
         ...,
         [  67.],
         [ 162.],
         [  -1.]],

        [[  -1.],
         [ 159.],
         [  -1.],
         ...,
         [  12.],
         [ 

KeyboardInterrupt: 

In [ ]:
model = LSTMRegressor(1,5,2,5).to(device)
num_epochs = 1000

for epoch in range(num_epochs):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        preds, _ = model(x_packed, (hidden, state))
        # preds = torch.transpose(preds, 0, 1)

        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

In [ ]:
true_preds, num_preds = 0., 0.

with torch.no_grad():
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[0])
        hidden, state = hidden.to(device), state.to(device)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        preds_packed, _ = model(x_packed, (hidden, state))
        preds, _ = model(x_packed, (hidden, state))
        # preds = torch.transpose(preds, 0, 1)

        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        preds = torch.argmax(preds, dim=1)
        targets = torch.argmax(targets, dim=1)
        print(preds)
        print(targets)
        true_preds += (preds == targets).sum().item()
        num_preds += targets.shape[0]
acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")